In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('../Data/selected_data/merged_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217242 entries, 0 to 217241
Data columns (total 31 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           217242 non-null  int64  
 1   batter               217242 non-null  object 
 2   bowler               217242 non-null  object 
 3   non_striker          217242 non-null  object 
 4   runs_by_bat          217242 non-null  int64  
 5   extra_runs           217242 non-null  int64  
 6   total_runs_delivery  217242 non-null  int64  
 7   current_team_total   217242 non-null  int64  
 8   runs_remain          217117 non-null  float64
 9   batter_runs          217242 non-null  int64  
 10  balls_faced          217242 non-null  int64  
 11  wickets_fallen       217242 non-null  int64  
 12  delivery             217242 non-null  int64  
 13  over                 217242 non-null  int64  
 14  extra_type           217242 non-null  object 
 15  wicket_type      

C:\Users\kaush\AppData\Local\Temp\ipykernel_32836\1736518648.py:6: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Data/selected_data/merged_data.csv')


In [2]:
# Drop the columns that are not needed
columns_needed = ['match_id', 'over', 'delivery', 'current_team_total', 'final_team_total', 'wickets_fallen', 'innings']
rrdata = data[columns_needed]

# Calculate the run rate
# run rate = total runs / total overs
# but during an over, the run rate is calculated as total runs / (total overs + delivey/6)
rrdata['run_rate'] = rrdata['current_team_total'] / (rrdata['over'] + rrdata['delivery']/6)
rrdata.head(20)

C:\Users\kaush\AppData\Local\Temp\ipykernel_32836\3483876993.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rrdata['run_rate'] = rrdata['current_team_total'] / (rrdata['over'] + rrdata['delivery']/6)


,match_id,over,delivery,current_team_total,final_team_total,wickets_fallen,innings,run_rate
0,Australia_Sri Lanka_2017-02-17,0,1,0,168,0,1,0.000000
1,Australia_Sri Lanka_2017-02-17,0,2,0,168,0,1,0.000000
2,Australia_Sri Lanka_2017-02-17,0,3,1,168,0,1,2.000000
3,Australia_Sri Lanka_2017-02-17,0,4,3,168,0,1,4.500000
4,Australia_Sri Lanka_2017-02-17,0,5,3,168,0,1,3.600000
5,Australia_Sri Lanka_2017-02-17,0,6,6,168,0,1,6.000000
6,Australia_Sri Lanka_2017-02-17,1,1,6,168,0,1,5.142857
7,Australia_Sri Lanka_2017-02-17,1,2,7,168,0,1,5.250000
8,Australia_Sri Lanka_2017-02-17,1,3,7,168,0,1,4.666667
9,Australia_Sri Lanka_2017-02-17,1,4,7,168,0,1,4.200000
